In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 14.2 합성곱층

In [ ]:
import numpy as np
from sklearn.datasets import load_sample_image

china = load_sample_image('china.jpg')/255.
flower = load_sample_image('flower.jpg')/255.

images = np.array([china,flower])

batch_size,height,width,channels = images.shape

images.shape

In [ ]:
#h,w,c,n
filters = np.zeros(shape=(7,7,channels,2),dtype=np.float32)

#수직필터. 너비=3
filters[:,3,:,0]=1
#수평필터. 높이=3
filters[3,:,:,1]=1

In [ ]:
outputs =tf.nn.conv2d(images,filters, strides=1, padding="SAME")
plt.imshow(outputs[0,:,:,1],cmap="gray")
#첫번째 이미지의 두번쨰(1)특성맵
plt.show()


In [ ]:
plt.imshow(outputs[1,:,:,1],cmap="gray")
#두번째 이미지의 두번째(1) 특성맵
plt.show()

케라스로 3x3필터 32개 만들기

In [ ]:
conv = keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                           padding="same", activation="relu")

## 14.3 풀링 층 

### 14.3.1 풀링 텐서플로구현

In [ ]:
max_pool = keras.layers.MaxPool2D(pool_size=2)

In [ ]:
avg_pool = keras.layers.AvgPool2D(pool_size=2)

깊이 풀링도 가능.

In [ ]:
#텐서플로 저수준 api 사용
output = tf.nn.max_pool(images,
                        ksize=(1,1,1,3),#배치,높이,너비,깊이
                        strides=(1,1,1,3),
                        padding="VALID")

In [ ]:
#케라스 모델에 쓰기 위해서는 람다 층으로 적용

depth_pool = keras.layers.Lambda(lambda X: tf.nn.max_pool(X,
                                                          ksize=(1,1,1,3),
                                                          strides=(1,1,1,3),
                                                          padding="valid"))

전역 평균 풀링(특성맵마다 하나의 숫자 출력)

In [ ]:
global_avg_pool = keras.layers.GlobalAveragePooling2D()

In [ ]:
#같은 값
global_avg_pool =keras.layers.Lambda(lambda X: tf.reduce_mean(X, axis=[1,2]))

# 14.4 CNN 구조

Fashion mnist를 위한 간단한 CNN 구조

In [ ]:
model = keras.models.Sequential([
                                 #필터 수, 커널 사이즈
        keras.layers.Conv2D(64,7, activation="relu",padding="same", 
                            input_shape=[28,28,1]),
                            #컬러채널이 한개 -->1
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(128,3,activation='relu',padding="same") ,
        keras.layers.Conv2D(128,3,activation='relu',padding="same") ,
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128,3,activation='relu',padding="same") ,
        keras.layers.Conv2D(128,3,activation='relu',padding="same") ,
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128,activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64,activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10,activation="softmax")
        
                              
])

# 14.5 케라스를 사용해 ResNet-34 CNN 구현하기

사용자 정의 층으로 residual unit 만들기

In [ ]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu",**kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    #main 
    self.main_layers=[
        keras.layers.Conv2D(filters,3,strides=strides,padding="same",use_bias=False),
        keras.layers.BatchNormalization(),
        self.activation,

        keras.layers.Conv2D(filters,3,strides=1,padding="same",use_bias=False),
        keras.layers.BatchNormalization()
    ]

    #skip 스킵연결 
    self.skip_layers= []

    #만약 스트라이드로 인해 특성맵이 줄어들면 입력의 크기도 맞춰 줄여줌.
    if strides>1:
      self.skip_layers = [
                          keras.layers.Conv2D(filters,1,strides=strides,
                                              padding="same",use_bias=False),
                          keras.layers.BatchNormalization()
      ]


  def call(self,inputs):
    Z= inputs

    for layer in self.main_layers:
      Z= layer(Z)
    
    skip_Z = inputs

    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)

    return self.activation(Z+skip_Z)


resisual unit 적용하기

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[224,224,3],
                              padding="same",use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3,strides=2, padding="same"))

prev_filters =64

#resnet-34 . 64 3개 128 4개 256 6개 512 3개
for filters in [64]* 3 + [128]* 4 +[256]* 6+[512]*3:
  strides = 1 if filters==prev_filters else 2
  model.add(ResidualUnit(filters,strides=strides))
  prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
model.summary()

# 14.6 케라스에서 제공하는 사전훈련된 모델 사용하기

resnet-50

In [ ]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")

사전훈련된 모델의 인풋 크기와 맞추기

In [ ]:
images.shape

In [ ]:
plt.imshow(images[0])

In [ ]:
images_resized = tf.image.resize(images,[224,224])

In [ ]:
plt.imshow(images_resized[0])

모델마다 다른 인풋범위를 원함.

0~1 --> 0 ~255로 바꿔줌

In [ ]:
images_resized[0]

In [ ]:
inputs =keras.applications.resnet50.preprocess_input(images_resized*255)

In [ ]:
inputs[0]

In [ ]:
y_proba = model.predict(inputs)

In [ ]:
y_proba.shape

In [ ]:
top_k = keras.applications.resnet50.decode_predictions(y_proba, top=3)

for image_index in range(len(images)):
  print("이미지 #{}".format(image_index))
  for class_id , name, y_proba in top_k[image_index]:
    print("{} - {:12s} {:.2f}%".format(class_id,name,y_proba*100))

  print()

# 14.7 사전훈련된 모델을 사용한 전이 학습

In [ ]:
import tensorflow_datasets as tfds

tfds 에서 제공하는 스플릿파라미터와 함게 tf dataset 로드하기

In [ ]:
(test_set_raw,valid_set_raw,train_set_raw),info = tfds.load("tf_flowers",split=["train[:10%]","train[10%:25%]","train[25%:]"],
                                         as_supervised=True,
                                         with_info=True
                                         )

In [ ]:
info

In [ ]:
dataset_size = info.splits["train"].num_examples
dataset_size

In [ ]:
class_names = info.features["label"].names
class_names

In [ ]:
n_classes = info.features["label"].num_classes
n_classes

In [ ]:
plt.figure(figsize=(12,10))


for index,(image,label) in enumerate(train_set_raw.take(9)):
  
  plt.subplot(3,3,index+1)
  plt.imshow(image)
  plt.title("class: {}".format(class_names[label]))
  plt.axis("off")

이 모델은 224x224 지원

In [ ]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [ ]:
batch_size=32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)



Xception, 가중치는 이미지넷에서 학습

avg pooling층과 fully-connected층을 떼고 모델 불러옴.

In [ ]:
base_model =keras.applications.xception.Xception(weights="imagenet",
                                                 include_top=False) 


In [ ]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for index, layer in enumerate(base_model.layers):
    print(index, layer.name)

초기 가중치 동결.

이후만 학습

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
train_set

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

val accuracy가 75~80 선에서 멈추면

이후에 모든 층을 낮은 학습률로 학습

In [ ]:
for layer in base_model.layers:
  layer.trainable=True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=1)
                    #오래 걸려서;; epochs=40)

# 14.8 분류와 위치 추정

In [ ]:
#수직 수평 너비 높이 4개의 좌표로 mse 만 추가하면됨.

base_model =keras.applications.xception.Xception(weights ="imagenet",
                                                 include_top=False)

avg = keras.layers.AveragePooling2D()(base_model.output)
#아웃풋을 두가지 만든다.

#기존 클래스 분류 아웃풋
class_output = keras.layers.Dense(n_classes,activation="softmax")(avg)
#위치 회귀 아웃풋
loc_output = keras.layers.Dense(4)(avg)

model = keras.Model(inputs=base_model.input,
                    outputs =[class_output,loc_output])
model.compile(loss=[keras.losses.sparse_categorical_crossentropy,
                    keras.losses.mean_squared_error],
              loss_weights =[0.8,0.2], # 어떤 가중치를 중요하게 옵타마이징 할것인가.
              optimizer= optimizer,
              metrics ="accuracy")


회귀를 위한 바운딩박스 레이블이 필요. 

없는경우 다양한 api 혹은 직접..

MSE가 아닌 iou도 가능

In [ ]:
iou_metric = keras.metrics.MeanIoU

# 14.9 객체 탐지

Yolo3에 대한 좋은 설명 

https://mickael-k.tistory.com/27 

## 14.10 시맨틱 분할

mask r-cnn

# 연습문제 9 Mnist CNN 모델

In [ ]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.mnist.load_data()

In [ ]:
X_train_full.shape

In [ ]:
X_train, X_valid ,y_train,y_valid = X_train_full[5000:],X_train_full[:5000],y_train_full[5000:],y_train_full[:5000]

케라스는 배치와 함께 움직이기 때문에 추가 차원 필요

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[...,np.newaxis]
X_test = X_test[...,np.newaxis]

In [ ]:
X_train= X_train/255.
X_valid = X_valid/255.
X_test = X_test/255.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Conv2D(128,7,
                                                     padding="same",                                              
                                                     activation="relu"),
                                 keras.layers.MaxPool2D(2),
                                 keras.layers.Conv2D(64,3,
                                                     padding="same",
                                                     activation="relu"),
                                 keras.layers.MaxPool2D(2),
                                 keras.layers.Conv2D(64,3,
                                                     padding="same",
                                                     activation="relu"),
                                 keras.layers.Flatten(),
                                 keras.layers.Dropout(0.25),
                                 keras.layers.Dense(10,activation='softmax')             
])


In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr=0.02),
              metrics = ["accuracy"]
            )

In [ ]:
model.fit(X_train,y_train,epochs=10,
          validation_data=(X_valid,y_valid))

In [ ]:
model.evaluate(X_test,y_test)

# 10. Use transfer learning for large image classification

a. 최소 100개의 이미지가 잇는 훈련세트 만들기.

b. 훈련, 검증, 테스트 세트로 나누기

In [ ]:
import tensorflow_datasets as tfds

10퍼 15퍼 75퍼

In [ ]:
datasets,info = tfds.load("tf_flowers",as_supervised=True,with_info=True,split=["train[:10%]","train[10%:25%]","train[25%:]"])

In [ ]:
info

#3컬러채널
#총 3670개

In [ ]:
test_data_raw=datasets[0]
valid_data_raw=datasets[1]
train_data_raw=datasets[2]

5개의클래스

In [ ]:
n_classes = info.features["label"].num_classes
n_classes

클래스이름

In [ ]:
classes = info.features["label"].names
classes

시각회

In [ ]:
plt.figure(figsize=(12,6))
for index,(image,label) in enumerate(train_data_raw.take(6)):

  plt.subplot(2,3,index+1)
  plt.axis("off")
  plt.title("#{} : {}".format(index,classes[label]))
  plt.imshow(image)

c. 적절한 전처리 연산과 선택적으로 데이터 증식을 수행하는 입력 파이프라인 만들기

In [ ]:
for x,y in train_data_raw.take(1):
  print(x)

preprocess_input 함수는 불러온 모델에 필요한 형식에 이미지를 적절하게 맞춰줌

In [ ]:
def preprocess(image,label):
  resized_image = tf.image.resize(image,[224,224])
  final_image = keras.applications.resnet50.preprocess_input(resized_image)
  return final_image,label

In [ ]:
train_data = train_data_raw.shuffle(1000).repeat()
train_data = train_data.map(preprocess).batch(32).prefetch(1)
valid_data = valid_data_raw.map(preprocess).batch(32).prefetch(1)
test_data = test_data_raw.map(preprocess).batch(32).prefetch(1)

In [ ]:
train_data

In [ ]:
valid_data

프리트레인 불러오기

In [ ]:
base_model = keras.applications.resnet50.ResNet50(include_top=False,weights="imagenet")

In [ ]:
base_model.summary()

In [ ]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer =keras.optimizers.Nadam(lr=0.01),
              metrics = ["accuracy"])

In [ ]:
history = model.fit(train_data,
                    steps_per_epoch=len(train_data_raw)//32,
                    epochs=10,
                    validation_data =valid_data
                    )

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer =keras.optimizers.Nadam(lr=0.001),
              metrics = ["accuracy"])

In [ ]:
history = model.fit(train_data,
                    steps_per_epoch=len(train_data_raw)//32,
                    epochs=10,
                    validation_data =valid_data
                    )

In [ ]:
model.evaluate(test_data)